In [43]:
%load_ext autoreload
%autoreload 2

import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision

from torch.utils.tensorboard import SummaryWriter
from functools import partial
from evolver import CrossoverType, MutationType, MatrixEvolver
from models.unet import UNet
from dataset_utils import PartitionType
from landcover_dataloader import LandCoverDataset, create_land_cover_dataset_from_config, get_land_cover_dataloader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)  

In [3]:
dataloader_params = {
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 6}

dataset_dir = os.path.join(os.getcwd(), "data/landcover_small")
create_land_cover_dataset_from_config(dataset_dir)
train_loader = get_land_cover_dataloader(dataset_dir, PartitionType.TRAIN, dataloader_params)
validation_loader = get_land_cover_dataloader(dataset_dir, PartitionType.VALIDATION, dataloader_params)
finetuning_loader = get_land_cover_dataloader(dataset_dir, PartitionType.FINETUNING, dataloader_params)
test_loader = get_land_cover_dataloader(dataset_dir, PartitionType.TEST, dataloader_params)

In [20]:
try:
    torch.cuda.empty_cache()
    del batch_x, batch_y, net
    gc.collect()
    
except:
    print("No data to garbage collect.")


In [33]:
# Define Model Loss and Optimizers
params = {
    'max_epochs': 10,
    'n_classes': 4,
    'in_channels': 4,
    'depth': 4,
    'learning_rate': 0.001,
    'momentum': 0.9,
    'report_steps': 10
}

In [35]:
model = UNet(in_channels = params['in_channels'],
           n_classes = params['n_classes'],
           depth = params['depth'])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=params['learning_rate'],
                            momentum=params['momentum'])

for epoch in range(params['max_epochs']):
    for i, data in enumerate(train_loader):
        batch_x, batch_y = data
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

In [51]:
from ignite.metrics import Accuracy, Loss, Precision, Recall
from ignite.engine import Engine

metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion),
    "precision": Precision(),
    "recall": Recall(),
}

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
train_evaluator.logger = setup_logger("Train Evaluator")
validation_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
validation_evaluator.logger = setup_logger("Validation Evaluator")

@trainer.on(Events.EPOCH_COMPLETED)
def compute_metrics(engine):
    train_evaluator.run(train_loader)
    validation_evaluator.run(validation_loader)

tb_logger = TensorboardLogger(log_dir=log_dir)
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=params['report_steps']),
    tag="training",
    output_transform=lambda loss: {"batchloss": loss},
    metric_names="all",
)

for tag, evaluator in [("training", train_evaluator), ("validation", validation_evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names=["loss", "accuracy"],
        global_step_transform=global_step_from_engine(trainer),
    )

tb_logger.attach_opt_params_handler(trainer, event_name=Events.ITERATION_COMPLETED(every=100), optimizer=optimizer)

tb_logger.attach(trainer, log_handler=WeightsScalarHandler(model),
                 event_name=Events.ITERATION_COMPLETED(every=params['report_steps']))
tb_logger.attach(trainer, log_handler=WeightsHistHandler(model),
                 event_name=Events.EPOCH_COMPLETED(every=params['report_steps']))
tb_logger.attach(trainer, log_handler=GradsScalarHandler(model),
                 event_name=Events.ITERATION_COMPLETED(every=params['report_steps']))
tb_logger.attach(trainer, log_handler=GradsHistHandler(model),
                 event_name=Events.EPOCH_COMPLETED(every=params['report_steps']))

def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint = ModelCheckpoint(
    log_dir,
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="validation_accuracy",
    global_step_transform=global_step_from_engine(trainer),
)
validation_evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})
trainer.run(train_loader, max_epochs=params['max_epochs'])
tb_logger.close()

Current run is terminating due to exception: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same.
Engine run is terminating due to exception: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same.


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same